# Day 2

In [ ]:
import pykx as kx

🍎 After assessing the environment, you realize that survival isn’t just about temperature — you also need to manage your food supply.

Your next task at your new shelter is to sort, clean, and organize the inventory.

📂 Data You’ve found a crate containing a list of foods. The crate has an inventory.txt file that looks like this:

```
`apples`bread`milk`carrots`eggs`yogurt`cheese`lettuce`tomatoes`grapes`banana
```

🧩 Challenge
- Load the inventory.txt into a kdb+ table.
- Classify each food item by adding a new column with a flag (1b,0b) indicating whether the food is perishable or not.
- Clean the inventory by: Removing any expired items (items with quantity of zero or less).
- Grouping any duplicated items (e.g., if milk appears twice, the resulting table should have only one row with the quantities summed).
- Update and replenish the inventory: After a recount, you need to update your inventory.
- Items to update: banana +5, rice +10, chocolate +3.
- Some foods are already in your list, others may be new. Overwrite quantities where the food exists, otherwise append new rows where they don’t.

In [ ]:
inventory:("SJ";1#",")0:`:inventory.txt;

In [ ]:
perishables: `apples`bread`milk`carrots`eggs`yogurt`cheese`lettuce`tomatoes`grapes`banana;
t1: delete from inventory where quantity<=0;
t2: select sum quantity by name from t1;

In [ ]:
new_items:([name:`banana`rice`chocolate];quantity:5 10 3);
new_inventory: t2 upsert new_items;

In [ ]:
ans1: count new_inventory;
ans2: exec sum quantity from new_inventory where name in perishables;
show (ans1; ans2)

## Extra exercise

Now that we know what food is available, we need to transport it to the base.

We found another file containing the month's weather forecast. `unreachable_hours.csv` is a file with a single column, `hour`, containing timestamps.

The hours listed in this file represent times when adverse weather will prevent us from reaching the ship.

To organize the supply runs, we need to schedule the departure times and the specific items to retrieve.

- One trip requires 1 hour.
- We can transport two types of food per trip. For example, on one trip we can bring all the onions and all the carrots, but we cannot add a single cookie.
- We must prioritize food that is about to expire.

Goal: Organize the list of food items to retrieve next week.

Tip:

To generate all the possible hours of the next week use `hours_of_next_week: asc raze (.z.d+til 7)+\:til[24]*01:00;`

Hint:

You may wan't to use the `exec` qsql keyword. Applying this to one table's column results in the list of values of that column.

usage example to obtain the values of a specific column:

```
exec example_col from example_table
```

In [ ]:
unreachable_shore: (1#"P"; 1#",") 0: `:unreachable_hours.csv;

hours_of_next_week: asc raze (.z.d+til 7)+\:til[24]*01:00;
max_trip_items: 2;

approachable_hours: ([] hour: hours_of_next_week) except unreachable_shore;
inventory_priorities: `perishable`quantity xdesc update perishable: name in perishables from new_inventory;

items_per_trip: exec name from select name by 2 xbar i from inventory_priorities;
update food_to_take: count[approachable_hours]#items_per_trip from approachable_hours

## Reference

[Q Datatypes](https://code.kx.com/q/basics/datatypes/)

[Q lists](https://code.kx.com/q4m3/3_Lists/)

[File handles](https://code.kx.com/q4m3/11_IO/#1111-file-handles)

[qsql](https://code.kx.com/q/basics/qsql/)

[Table construction](https://code.kx.com/q/kb/faq/#table-notation)

[.z.d](https://code.kx.com/q/ref/dotz/#zt-zt-zd-zd-timedate-shortcuts) - Current date

[Virtual column i](https://code.kx.com/q/basics/qsql/#virtual-column-i)

Operators:
- [0:](https://code.kx.com/q/ref/file-text/#load-csv) - File reading
- [#](https://code.kx.com/q/ref/take/) - Take
- [sum](https://code.kx.com/q/ref/sum/)
- [upsert](https://code.kx.com/q/ref/upsert/)
- [til](https://code.kx.com/q/ref/til/) - List generation
- [\\:](https://code.kx.com/q/ref/maps/#each-left-and-each-right) - Each left
- [raze](https://code.kx.com/q/ref/raze/)
- [asc](https://code.kx.com/q/ref/asc/)
- [except](https://code.kx.com/q/ref/except/)
- [in](https://code.kx.com/q/ref/in/)
- [xdesc](https://code.kx.com/q/ref/desc/#xdesc)
- [xbar](https://code.kx.com/q/ref/xbar/)